<a href="https://colab.research.google.com/github/sssharaf/ml-nlp/blob/master/start_servers_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/gdrive')
!ls -ltr /gdrive/'My Drive'/ML/data/start-servers-play
!pip install pytorch_transformers
!ln -s  /gdrive/'My Drive'/ML/data/start-servers-play data
!ls -ltr data/*

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
total 2
-rw------- 1 root root 1339 Aug 10 17:21 train.csv
-rw------- 1 root root  437 Aug 10 17:24 val.csv
ln: failed to create symbolic link 'data/start-servers-play': Function not implemented
-rw------- 1 root root 1339 Aug 10 17:21 data/train.csv
-rw------- 1 root root  437 Aug 10 17:24 data/val.csv


In [0]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
import pytorch_transformers as pt
from pytorch_transformers import BertTokenizer, BertConfig,BertForMaskedLM,BertModel,DistilBertTokenizer, DistilBertModel,DistilBertForSequenceClassification 
import os
import typing
from typing import Dict,List,Sequence,Set
from types import SimpleNamespace as SN
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight,compute_sample_weight
T_BertTokenizer = typing.NewType("BertTokenizer",BertTokenizer)
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

In [16]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

100%|██████████| 231508/231508 [00:00<00:00, 916029.35B/s]


In [0]:
trn_df = pd.read_csv('data/train.csv',dtype={'action':'category','component':'category'})
val_df = pd.read_csv('data/val.csv',dtype={'action':'category','component':'category'})

In [197]:
a_ohe = OneHotEncoder(sparse=False)
a_ohe.fit(trn_df.loc[:,['action']])
print(a_ohe.transform([['start']]))
print(a_ohe.categories_[0])
c_ohe = OneHotEncoder(sparse=False)
c_ohe.fit(trn_df.loc[:,['component']])

action_le = LabelEncoder()
action_le.fit(trn_df.action)
component_le = LabelEncoder()
component_le.fit(trn_df.component)

ValueError: ignored

In [0]:
class MyDataset(Dataset):

    def __init__(self,df:DataFrame,max_len = 16):
        self.df = df
        self.max_len=max_len
        self.action = self.df.action.cat.codes
        self.component = self.df.component.cat.codes

    def __getitem__(self,index):
        X = self.df.iloc[index]['comment_text']
        X = f"[CLS] {X}[SEP]"
        encoded = torch.tensor(tokenizer.encode(X),dtype=torch.long)
        X = torch.zeros(self.max_len,dtype=torch.long)
        X[:len(encoded)] = encoded
        X[len(encoded)+1:] = torch.tensor(tokenizer.pad_token_id,dtype=torch.long)
        X_attn_mask = X!=tokenizer.pad_token_id
        Y1 = self.df.iloc[index]['action']
        Y1 = action_le.transform([Y1])
        #Y1 = a_ohe.transform([[Y1]])
        Y1 = torch.tensor(Y1,dtype=torch.long)
        Y2 = self.df.iloc[index]['component']
        Y2 = component_le.transform([Y2])
        #Y2 = c_ohe.transform([[Y2]])
        Y2 = torch.tensor(Y2, dtype=torch.long)
        return (X,X_attn_mask),(Y1.squeeze(),Y2.squeeze())

    def __len__(self):
        return len(self.df)

    def components(self):
        return self.component

In [220]:
trn_ds = MyDataset(trn_df)
val_ds = MyDataset(val_df)
trn_ds[2]

((tensor([  101,  2644,  1996,  3119, 13151,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0]),
  tensor([ True,  True,  True,  True,  True,  True, False, False, False, False,
          False, False, False, False, False, False])),
 (tensor(1), tensor(1)))

In [0]:
trn_dl = DataLoader(dataset=trn_ds,batch_size=4,pin_memory=True)
val_dl = DataLoader(dataset=val_ds,batch_size=4,pin_memory=True)

In [0]:
# Model
class MyModel(nn.Module):
  def __init__(self, freeze_bert = True):
    super().__init__()
    self.bert_lyr = BertModel.from_pretrained('bert-base-uncased')
    self.action_cls_lyr = nn.Sequential(
        nn.Dropout(0.1),
        nn.Linear(768,len(a_ohe.categories_[0]),bias=False)
    )
    self.component_cls_lyr = nn.Sequential(
                nn.Dropout(0.1),
                nn.Linear(768,len(c_ohe.categories_[0]),bias=False)
    )
    

    #Freeze bert layers
    if freeze_bert:
        for p in self.bert_lyr.parameters():
            p.requires_grad = False
    #nn.init.xavier_uniform_(self.action_cls_lyr.weight)
    #nn.init.xavier_uniform_(self.component_cls_lyr.weight)

  def forward(self, seq, attn_masks):
    seq_emb,ctx = self.bert_lyr(seq,attention_mask =attn_masks)
    return self.action_cls_lyr(ctx),self.component_cls_lyr(ctx)

model = MyModel()

In [241]:
print(f'Device type is {DEVICE.type}')
model.to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-3, )
action_criterion = nn.modules.loss.CrossEntropyLoss(reduction='mean')
component_criterion = nn.modules.loss.CrossEntropyLoss(reduction='mean')

n_epochs = 1000

def evaluate_model(model:MyModel,dl:DataLoader,optimizer):
  t_loss=0
  for (X, attn_mask),(Y1,Y2) in dl:
    X , attn_mask,Y1,Y2 = X.to(DEVICE),attn_mask.to(DEVICE),Y1.to(DEVICE),Y2.to(DEVICE)
    p_a,p_c = model(X,attn_mask)
    loss = action_criterion(p_a,Y1) + component_criterion(p_c,Y2)
    if optimizer is not None:
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    t_loss += loss.item()
  return t_loss

for epoch in range(n_epochs):
  t_loss=0
  t_loss = evaluate_model(model,trn_dl,optimizer)
  v_loss = 0
  with torch.no_grad():
    v_loss = evaluate_model(model,val_dl,None)
  # for (X, attn_mask),(Y1,Y2) in trn_dl:
  #   X , attn_mask,Y1,Y2 = X.to(DEVICE),attn_mask.to(DEVICE),Y1.to(DEVICE),Y2.to(DEVICE)
  #   p_a,p_c = model(X,attn_mask)
  #   loss = action_criterion(p_a,Y1) + component_criterion(p_c,Y2)
  #   optimizer.zero_grad()
  #   loss.backward()
  #   optimizer.step()
  #   t_loss += loss.item()
  print(f'Epoch:{epoch} Training loss={t_loss} , Validation loss:{v_loss}')


Device type is cuda
Epoch:0 Training loss=33.82660889625549 , Validation loss:6.631506323814392
Epoch:1 Training loss=19.851772665977478 , Validation loss:6.9083579778671265
Epoch:2 Training loss=23.31220269203186 , Validation loss:6.341805577278137
Epoch:3 Training loss=23.073599219322205 , Validation loss:6.081575989723206
Epoch:4 Training loss=21.743334650993347 , Validation loss:6.24396014213562
Epoch:5 Training loss=21.57986319065094 , Validation loss:6.531764507293701
Epoch:6 Training loss=20.794257164001465 , Validation loss:6.160957455635071
Epoch:7 Training loss=20.943405270576477 , Validation loss:6.282789826393127
Epoch:8 Training loss=20.567913055419922 , Validation loss:6.126418590545654
Epoch:9 Training loss=20.280345916748047 , Validation loss:5.952383399009705
Epoch:10 Training loss=19.91410493850708 , Validation loss:6.207619547843933
Epoch:11 Training loss=19.423296689987183 , Validation loss:5.8740071058273315
Epoch:12 Training loss=19.606853485107422 , Validation lo

In [212]:
action_criterion(p_a,Y1)

RuntimeError: ignored